# String distance

In [1]:
import nltk
import numpy as np
import pandas as pd

In [2]:
words = nltk.corpus.words.words()

In [3]:
pd.DataFrame(words).to_csv("words_nltk.csv")

## Assuming same cost for all edits

In [4]:
def create_memoization_table(X,Y):
    len_x = len(X)
    len_y = len(Y)
    D = np.zeros((len_x+1,len_y+1), dtype=np.int32)
    
    for i in range(len(X)+1):
        for j in range(len(Y)+1):

            if i == 0:
                D[i][j] = j    

            elif j == 0:
                D[i][j] = i  

            elif X[i-1] == Y[j-1]: 
                D[i][j] = D[i-1][j-1]

            else:
                D[i][j] = 1+min(D[i][j-1],      # Insert 
                                D[i-1][j],      # Remove 
                                D[i-1][j-1])    # Replace 
    return D

In [5]:
x = "EXPONENTIAL"
y = "POLYNOMIAL"
D = create_memoization_table(x,y)
D[-1,-1]

6

In [6]:
x = "Elliot"
y = "Elia"
D = create_memoization_table(x,y)
D[-1,-1]

3

In [7]:
x = "hi"
y = "hill"
D = create_memoization_table(x,y)
print("\nThe distance between {} and {} is {}".format(x,y,D[-1,-1]))


The distance between hi and hill is 2


##### Timing implementation

In [8]:
x = "EXPONENTIAL"
y = "POLYNOMIAL"

In [9]:
def edit_distance_fast(x,y):
    D = create_memoization_table(x,y)
    return D[-1,-1]

In [10]:
%%timeit
edit_distance_fast(x,y)

199 µs ± 45.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [11]:
%%timeit
nltk.edit_distance(x,y)

159 µs ± 32.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
nltk.edit_distance(x,y) == edit_distance_fast(x,y)

True

### Different costs per operation

In [13]:
def memoization_table_weighted(X,Y):
    len_x = len(X)
    len_y = len(Y)
    D = np.zeros((len_x + 1, len_y + 1), dtype=np.int32)
    D[:,0] = range(len_x + 1)
    D[0,:] = range(len_y + 1)

    w_sub = 1
    w_del = 1
    w_ins = 1

    for i in range(1, len_x + 1):
        for j in range(1, len_y + 1):
            del_char = D[i-1,j] + w_del
            ins_char = D[i,j-1] + w_ins

            if X[i-1] == Y[j-1]:
                Z = 0
            else:
                Z = w_sub
            sub_char = D[i-1,j-1] + Z

            D[i,j] = min(del_char, ins_char, sub_char)

    return D

In [14]:
x = "Elliot"
y = "Elia"
D = memoization_table_weighted(x, y)
print("\nThe distance between {} and {} is {}".format(x,y,D[-1,-1]))


The distance between Elliot and Elia is 3


# Speeding up code

Simple example with cython

In [15]:
%load_ext cython

In [16]:
def fib(n):
    a = 0.
    b = 1.
    for i in range(n):
        a, b = a + b, a
    return a

In [17]:
%%cython --annotate
def cy_fib(int n):
    cdef int i
    cdef float a=0.0, b=1.0
    for i in range(n):
        a, b = a + b, a
    return a

Content of stdout:
_cython_magic_041ddae55551dc41dc1b15174d50ab60fef84cd7.c
   Creating library C:\Users\mecheste\.ipython\cython\Users\mecheste\.ipython\cython\_cython_magic_041ddae55551dc41dc1b15174d50ab60fef84cd7.cp39-win_amd64.lib and object C:\Users\mecheste\.ipython\cython\Users\mecheste\.ipython\cython\_cython_magic_041ddae55551dc41dc1b15174d50ab60fef84cd7.cp39-win_amd64.exp
Generating code
Finished generating code

In [18]:
fib(10)

55.0

In [19]:
cy_fib(10)

55.0

In [20]:
import timeit

n_times = 100000
t_fib = timeit.timeit("fib(10)", setup="from __main__ import fib",number=n_times)
t_cyfib = timeit.timeit("cy_fib(10)", setup="from __main__ import cy_fib",number=n_times)
t_fib_unit = t_fib/n_times

t_cyfib      = timeit.timeit("cy_fib(10)", setup="from __main__ import cy_fib",number=n_times)
t_cyfib_unit = t_cyfib/n_times

print(" Python version took: {} sec\n Cython version took: {} sec\n Cython is {:.0f}x faster"\
      .format(t_fib, t_cyfib, t_fib/t_cyfib))

print("\n Python version 1 run took: {} sec\n Cython version 1 run took: {} sec\n Cython is {:.0f}x faster"\
      .format(t_fib_unit, t_cyfib_unit, t_fib_unit/t_cyfib_unit))

 Python version took: 0.08786199999999766 sec
 Cython version took: 0.00580539999999985 sec
 Cython is 15x faster

 Python version 1 run took: 8.786199999999767e-07 sec
 Cython version 1 run took: 5.8053999999998496e-08 sec
 Cython is 15x faster


### Speeding up edit distance



##### Exercise fill in cy_create_memoization_table so that it returns the matrix filled to compute the edit distance

In [21]:
%%cython --annotate

import numpy as np

def cy_create_memoization_table(str X, str Y):
    cdef int i, j, del_char, ins_char, sub_char, Z
    cdef int len_x = len(X)
    cdef int len_y = len(Y)
    cdef int [:, :] D =  np.zeros((len_x + 1, len_y + 1), dtype=np.int32)

    for i in range(len_x+1):
        D[i,0] = i

    for j in range(len_y+1):
        D[0,j] = j

    for i in range(1, len_x + 1):
        for j in range(1, len_y + 1):
            del_char = D[i-1,j] + 1
            ins_char = D[i,j-1] + 1

            if X[i-1] == Y[j-1]:
                Z = 0
            else:
                Z = 1
            sub_char = D[i-1,j-1] + Z

            D[i,j] = min(del_char, ins_char, sub_char)
    
    return D

Content of stdout:
_cython_magic_1fd7b7ccfaab019fb57a918d50d8e9d0fc2be56f.c
C:\Users\mecheste\.ipython\cython\_cython_magic_1fd7b7ccfaab019fb57a918d50d8e9d0fc2be56f.c(1467): warning C4005: '__pyx_nonatomic_int_type': macro redefinition
C:\Users\mecheste\.ipython\cython\_cython_magic_1fd7b7ccfaab019fb57a918d50d8e9d0fc2be56f.c(1418): note: see previous definition of '__pyx_nonatomic_int_type'
C:\Users\mecheste\.ipython\cython\_cython_magic_1fd7b7ccfaab019fb57a918d50d8e9d0fc2be56f.c(17030): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
C:\Users\mecheste\.ipython\cython\_cython_magic_1fd7b7ccfaab019fb57a918d50d8e9d0fc2be56f.c(17044): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
   Creating library C:\Users\mecheste\.ipython\cython\Users\mecheste\.ipython\cython\_cython_magic_1fd7b7ccfaab019fb57a918d50d8e9d0fc2be56f.cp39-win_amd64.lib and object C:\Users\mecheste\.ipython\cython\Users\mecheste\.ipython\cython\_cython_mag

In [22]:
D1 = create_memoization_table(x,y)
D1

array([[0, 1, 2, 3, 4],
       [1, 0, 1, 2, 3],
       [2, 1, 0, 1, 2],
       [3, 2, 1, 1, 2],
       [4, 3, 2, 1, 2],
       [5, 4, 3, 2, 2],
       [6, 5, 4, 3, 3]])

In [23]:
D2 = cy_create_memoization_table(x,y)
D2 = np.asarray(D2)

In [24]:
D2

array([[0, 1, 2, 3, 4],
       [1, 0, 1, 2, 3],
       [2, 1, 0, 1, 2],
       [3, 2, 1, 1, 2],
       [4, 3, 2, 1, 2],
       [5, 4, 3, 2, 2],
       [6, 5, 4, 3, 3]])

In [25]:
t_create_memoization_table = timeit.timeit("x='exponential'; y='polynomial'; create_memoization_table(x,y)",
                                           setup="import numpy as np; from __main__ import create_memoization_table",
                                           number=5000)

In [26]:
t_cy_create_memoization_table = timeit.timeit("x='exponential'; y='polynomial'; cy_create_memoization_table(x,y)",
                                              setup="from __main__ import cy_create_memoization_table",
                                              number=5000)

In [27]:
t_nltk = timeit.timeit("x='exponential'; y='polynomial'; nltk.edit_distance(x,y)",
                        setup="import nltk ",
                        number=5000)

In [28]:
print(""" 
      Python version took: {} sec
      Cython version took: {} sec
      nltk   version took: {} sec
      Cython is {:.0f}x faster than python
      Cython is {:.0f}x faster than nltk
      """\
      .format(t_create_memoization_table, 
              t_cy_create_memoization_table,
              t_nltk, 
              t_create_memoization_table/t_cy_create_memoization_table,
              t_nltk/t_cy_create_memoization_table))


 
      Python version took: 1.1464116000000004 sec
      Cython version took: 0.016989900000002223 sec
      nltk   version took: 0.7099069999999976 sec
      Cython is 67x faster than python
      Cython is 42x faster than nltk
      


In [29]:
def edit_distance(x,y):
    return cy_create_memoization_table(x,y)[-1,-1]

### Return to the experiment where we computed closest word


##### Exercise: Return the last component of the DynamicProgramming matrix containing the edit distance

In [30]:
import editdistance

In [31]:
words = nltk.corpus.words.words()
len(words)

236736

In [32]:
%%time
mistake = "drauing" 
distances = []
for word in words:
    ed = nltk.edit_distance(mistake, word)
    distances.append(ed)
    
print("\nthe closest word is", words[np.argmin(distances)])


the closest word is drawing
CPU times: total: 20.1 s
Wall time: 21.7 s


In [33]:
%%time
mistake = "drauing" 
cy_distances = []
for word in words:
    ed = editdistance.eval(mistake, word)
    cy_distances.append(ed)
    
print("\nthe closest word is", words[np.argmin(cy_distances)])


the closest word is drawing
CPU times: total: 531 ms
Wall time: 571 ms


In [34]:
%%time
mistake = "drauing" 
cy_distances = []
for word in words:
    ed = edit_distance(mistake, word)
    cy_distances.append(ed)
    
print("\nthe closest word is", words[np.argmin(cy_distances)])


the closest word is drawing
CPU times: total: 688 ms
Wall time: 712 ms


In [35]:
distances == cy_distances

True

In [36]:
editdistance.eval("hi", "hi"), edit_distance("hi","hi")

(0, 0)

In [37]:
editdistance.eval("hi", "ho"), edit_distance("hi","ho")

(1, 1)

##### Interesting material on string similarities

Approximate string matching:

https://medium.com/@wolfgarbe/fast-approximate-string-matching-with-large-edit-distances-in-big-data-2015-9174a0968c0b

Levenshtein distance using a trie:

http://stevehanov.ca/blog/?id=114

About jaccard distance:

https://python.gotrained.com/nltk-edit-distance-jaccard-distance/